In [ ]:
%matplotlib widget
from gekko import GEKKO
import numpy as np
import pandas as pd
from random import random
import matplotlib.pyplot as plt
import requests
plt.ion()

#For Video Export

# Make an MP4 animation?
make_mp4 = True
if make_mp4:
    import imageio  # required to make animation
    import os
    try:
        os.mkdir('./figures')
    except:
        pass

# Get CO2 Data

In [ ]:
# Get CO2 Data
response = requests.get(url='https://api.energidataservice.dk/dataset/CO2EmisProg', params={"start": "2024-01-01T00:00","end":"2024-01-17T00:00","filter":"{\"PriceArea\":[\"DK2\"]}","sort":"Minutes5UTC ASC"})
print(f"request url: {response.request.url}")
print(f"status code: {response.status_code}", end="\n\n")

if response.ok:
	result = response.json()
	df_co2 = pd.DataFrame(result["records"])	
else:
	df_co2 = pd.DataFrame()
	raise f"Returned status {response.status_code}: {response.text}"

plt.figure("co2")
ax = plt.subplot()
df_co2['Minutes5DK'] = pd.to_datetime(df_co2['Minutes5DK'])

plt.plot(df_co2["CO2Emission"],color='orange',linewidth=1)

plt.xlabel('time')
plt.ylabel('g/kwh')

#plt.xticks(rotation=90)
plt.show()  




# Inflow Estimation

In [ ]:

df_Qin = pd.read_pickle("./data/pump_station_data_estimated.pkl")    
#df_Qin = pd.Series(df_Qin[0][1200:], name="Qin_hat")
Qin_v = df_Qin[0][1200::1000].values
plt.figure()
Qin_v[75:100] = 1500
plt.plot(Qin_v[:300],color='orange',linewidth=1)


# Setup the MPC

In [ ]:
h_hist = []
w_hist = []
Qout_hist = []
Qin_hist = []
P_hist = []
c_hist = []
emissions_hist = []

m = GEKKO(remote=False)
m.options.SOLVER = 1 # APOPT solver
m.TIME_SHIFT = 1

#time horizon
Thor = int(2*60) #sec
Ts = 1 #sec
n = Ts*Thor
m.time = np.linspace(0,int(Thor-1),n)

# Manipulated variable
w = m.MV(value=0, lb = 0, ub = 1500,integer=False)
w.STATUS = 1
w.DCOST = 0.07


# Controlled Variable
Qout = m.CV(value=0)
P = m.CV(value=0)
h = m.CV(value=170)



m.options.CV_TYPE = 1 # Linear error with deadband
eH = m.CV(value=0)
eH.SPHI=0         #Set point high for linear error model.
eH.WSPHI=10000    #Objective function weight on upper set point for linear error model.
eH.WSPLO=0        # Objective function weight on lower set point for linear error model
eH.STATUS =1      # eH : Error is considered in the objective function.

eL = m.CV(value=0)
eL.SPLO=0
eL.WSPHI=10     # key weight influencing attraction to lower levels
eL.WSPLO=15
eL.STATUS = 1    


c = m.Param()
m.Minimize(c*P)

Qin = m.Param()


# Process model
A_power = np.array([[3.7971E-01], \
                    [5.1151E-02], \
                    [3.1656E-01]])
A_outflow = np.array([[1.2995], \
                      [-4.3918E-01], \
                      [4.5231E-02], \
                      [0], \
                      [1.0387E-02]])

# B (ny x (nb x nu))
B_power = np.array([[[5.5404E-02], [-1.4118E-02], [-2.3452E-02]]]).T
B_outflow = np.array([[[7.1049E-02], [4.1327E-02], [6.4801E-02], [0], [-6.6199E-02]]]).T

# C_power = np.array([-6.6831])
# C_outflow = np.array([-7.3292E+01])

C_power = np.array([0])
C_outflow = np.array([0])



p_power = {'a':A_power,'b':B_power,'c':C_power}
p_outflow = {'a':A_outflow,'b':B_outflow,'c':C_outflow}




m.arx(p_power,P,w)
m.arx(p_outflow,Qout,w)


m.Equation(h.dt() == 1/18 *(Qin - Qout)/3600*100)
#m.Equation(h.dt() == 1/18 *(75 - m.if3(w-750,0,Qout))/3600*100)
emissions = m.Var()
m.Equation(emissions == m.integral(c/3600*P))
m.Equations([eH==h-200,eL==h-120]) # Errors

m.options.IMODE = 4 # Simulation



plt.figure(figsize=(15,10))
plt.show()
#plt.figure(3)


tm = []
simtime = 250
for i in range(0,simtime):


    # Set CO2 Prognosis
    c.value = df_co2["CO2Emission"][i:i+n]

    
    m.solve(disp=True)
    print(f"Solved {i+1}/{simtime} in {m.options.SOLVETIME}s")
    # Set Initialize RPM to previos value

    limitH = 180
    limitL = 140

    if(h.MODEL>=limitH):
        w.MEAS = 1500
    elif (h.MODEL<=limitL):
        w.MEAS = 0

    
    Qin.VALUE = Qin_v[i]

    # Appending to History
    w_hist.append(w.VALUE[0])
    h_hist.append(h.VALUE[0])
    Qout_hist.append(Qout.VALUE[0])
    Qin_hist.append(Qin_v[i])
    P_hist.append(P.VALUE[0])
    c_hist.append(c.VALUE[0])
    emissions_hist.append(emissions.VALUE[0])
    tm.append(i)
    # ------- Plotting -------------
    plt.clf()    
    ax1 = plt.subplot(4,1,1)
    plt.title("Moving Horizon MPC")
    plt.plot(m.time+i,w.Value,"k--",label='Pump Speed (Pred.)')
    plt.plot(tm,w_hist,"k-",label='Pump Speed (Meas.)')
    plt.plot([i,i],[ax1.get_ylim()[0],ax1.get_ylim()[1]],'k-',lw=1)
    plt.ylabel('Speed [RPM]')
    ax12 = ax1.twinx()
    ax12.plot(m.time+i,P.Value,"b--",label='Power (Pred.)')
    ax12.plot(tm,P_hist,"b-",label='Power (Meas.)') 
    ax12.set_ylabel('Power [kW]')
    ax12.tick_params(axis ='y', labelcolor = 'blue')
    h1, l1 = ax1.get_legend_handles_labels()
    h12, l12 = ax12.get_legend_handles_labels()
    ax1.legend(h1+h12, l1+l12, loc=5) 

    ax2 = plt.subplot(4,1,2, sharex=ax1)
    plt.plot(m.time+i,h.Value,"r--",label='Water Level (Pred.)')
    plt.plot(tm,h_hist,"r-",label='Water Level (Meas.)')
    plt.plot([0,i+n-1],[200,200],'k:',label='Upper Limit')
    plt.plot([0,i+n-1],[120,120],'k:',label='Lower Limit')
    plt.plot([i,i],[ax2.get_ylim()[0],ax2.get_ylim()[1]],'k-',lw=1)
    plt.legend(loc=5)
    plt.ylabel('Level [m]')
    plt.ylim([100, 220])
    

    ax3 = plt.subplot(4,1,3, sharex=ax1)
    plt.plot(m.time+i,Qout.Value,"r--",label='Outflow (Pred.)')
    plt.plot([(m.time+i)[0],(m.time+i)[-1]],[Qin_v[i], Qin_v[i]],"b--",label='Inflow (Pred.)')
    plt.plot(tm,Qout_hist,"r-",label='Outflow (Meas.)')
    plt.plot(tm,Qin_hist,"b-",label='Inflow (Est.)')
    plt.plot([i,i],[ax3.get_ylim()[0],ax3.get_ylim()[1]],'k-',lw=1)
    plt.legend(loc=5)
    plt.ylabel('Flow Rate [m^3/h]')

    ax4 = plt.subplot(4,1,4, sharex=ax1)
    plt.plot(m.time+i,c.Value,"k--",label='CO2 Market (Prog.)')
    plt.plot(tm,c_hist,"k-") 
    plt.plot([i,i],[ax4.get_ylim()[0],ax4.get_ylim()[1]],'k-',lw=1)
    plt.ylabel('CO2 Emission Prog. [g/kwh]')
    plt.xlabel('Time [s]')
    ax42 = ax4.twinx()
    ax42.plot(m.time+i,emissions.Value,"b--",label='Emissions (Pred.)')
    ax42.plot(tm,emissions_hist,"b-",label='Emissions (Meas.)') 
    ax42.set_ylabel('Emitted CO2 [g]')
    ax42.tick_params(axis ='y', labelcolor = 'blue') 
    h4, l4 = ax4.get_legend_handles_labels()
    h42, l42 = ax42.get_legend_handles_labels()
    ax4.legend(h4+h42, l4+l42, loc=5) 
    ax42.text(i,emissions.VALUE[0],str(int(emissions.VALUE[0]))+" g ",ha="right")

    plt.draw()

    if make_mp4:
        filename='./figures/plot_'+str(i+10000)+'.png'
        plt.savefig(filename)

    
print(f"Emitted CO2 over {i}s is {emissions_hist[-1]}")

In [ ]:


# generate mp4 from png figures in batches of 350
if make_mp4:

    path = "./figures/plot"   
    fileList = []
    for i in range(0,simtime-1):
        complete_path = f"{path}_{i+10000}.png"
        fileList.append(complete_path)

    writer = imageio.get_writer('results.mp4', fps=10)
    for im in fileList:
        writer.append_data(imageio.imread(im))
    writer.close()
